In [32]:
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough 
from langchain_community.vectorstores import FAISS 
from langchain_core.output_parsers import StrOutputParser 
from langchain_community.document_loaders import WebBaseLoader 
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.memory import ConversationBufferMemory 
import os
from dotenv import load_dotenv

In [33]:
load_dotenv()

True

In [34]:
# Definir URLs de las páginas a cargar
page_urls = [
   "https://www.bloomberglinea.com/latinoamerica/argentina/"
    #"https://www.ambito.com/finanzas"
    # "https://newsapi.org/v2/everything"
     ]

In [35]:
loader = WebBaseLoader(web_path=[page_urls])

In [36]:
# Cargar documentos desde las URLs
docs = []
for page_url in page_urls:
    try:
        loader = WebBaseLoader(web_path=[page_url])
        docs_lazy = loader.lazy_load()
        for doc in docs_lazy:
            docs.append(doc)
    except Exception as e:
        st.error(f"Error al cargar contenido de {page_url}: {e}")

In [37]:
docs

[Document(metadata={'source': 'https://www.bloomberglinea.com/latinoamerica/argentina/', 'title': 'Últimas noticias de la economía argentina: Dólar blue, MEP, oficial,  inflación, negocios, finanzas y  más - Bloomberg Línea', 'description': 'Todo sobre la economía de Argentina y sus noticias. La inflación el dólar blue, Mep, oficial, las finanzas y los negocios. La economía de Argentina en Bloomberg Línea', 'language': 'es'}, page_content='Últimas noticias de la economía argentina: Dólar blue, MEP, oficial,  inflación, negocios, finanzas y  más - Bloomberg Línea\n\n\n\n\n\n\nNoticia Últimas noticias de la economía argentina: Dólar blue, MEP, oficial,  inflación, negocios, finanzas y  más - Bloomberg LíneaÚltimas noticias económicas de LatamPUBLICIDADSeleccione un paísSuscribirseIngresarIdiomaEspañolEnglishPortuguêsNorteaméricaEEUUMéxicoCentroamérica y el CaribeCosta RicaEl SalvadorCaribeNicaraguaGuatemalaHondurasPanamáRepública DominicanaSudaméricaArgentinaBoliviaBrasilChileColombiaEcu

In [38]:
 #Dividir documentos en trozos más pequeños
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n", "\n\n", "."]
)

In [39]:
documents = splitter.split_documents(docs)

In [40]:
documents[0].metadata

{'source': 'https://www.bloomberglinea.com/latinoamerica/argentina/',
 'title': 'Últimas noticias de la economía argentina: Dólar blue, MEP, oficial,  inflación, negocios, finanzas y  más - Bloomberg Línea',
 'description': 'Todo sobre la economía de Argentina y sus noticias. La inflación el dólar blue, Mep, oficial, las finanzas y los negocios. La economía de Argentina en Bloomberg Línea',
 'language': 'es'}

In [41]:
documents

[Document(metadata={'source': 'https://www.bloomberglinea.com/latinoamerica/argentina/', 'title': 'Últimas noticias de la economía argentina: Dólar blue, MEP, oficial,  inflación, negocios, finanzas y  más - Bloomberg Línea', 'description': 'Todo sobre la economía de Argentina y sus noticias. La inflación el dólar blue, Mep, oficial, las finanzas y los negocios. La economía de Argentina en Bloomberg Línea', 'language': 'es'}, page_content='Últimas noticias de la economía argentina: Dólar blue, MEP, oficial,  inflación, negocios, finanzas y  más - Bloomberg Línea'),
 Document(metadata={'source': 'https://www.bloomberglinea.com/latinoamerica/argentina/', 'title': 'Últimas noticias de la economía argentina: Dólar blue, MEP, oficial,  inflación, negocios, finanzas y  más - Bloomberg Línea', 'description': 'Todo sobre la economía de Argentina y sus noticias. La inflación el dólar blue, Mep, oficial, las finanzas y los negocios. La economía de Argentina en Bloomberg Línea', 'language': 'es'}

In [42]:
llm=ChatGroq(model="llama-3.2-90b-vision-preview")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


c:\chatbot\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [43]:
vectorstore = FAISS.from_documents(documents, embeddings)

In [44]:
retriever = vectorstore.as_retriever()

In [45]:
# Configurar el prompt y el parser de salida
template = """Responder las preguntas  basado en el siguiente contexto:
{context}

Pregunta: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [46]:
output_parser = StrOutputParser()

In [47]:
#Configurar memoria para el agente
memory = ConversationBufferMemory()

In [48]:
# Crear la cadena de procesamiento
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [53]:
# Invocar la cadena con la memoria
question = "informacion sobre bitcoin"
response = chain.invoke(question)

# Guardar el contexto en la memoria
memory.save_context({"question": question}, {"response": response})

# Mostrar el contenido de la memoria
print(memory.buffer)  # Mostrar el historial de la conversación


Human: En forma de lista dame los titulares sobre finanzas
AI: A continuación, te proporciono una lista de titulares relacionados con finanzas:

1. Dólar hoy 22 de noviembre: Peso pierde terreno tras datos de inflación y PIB en México
2. Dólar hoy 21 de noviembre: Peso mexicano encabeza pérdidas entre las mayores divisas
3. UU. sanciona a Gazprombank para restringir exportaciones de gas ruso
4. Fitch Ratings mantiene a República Dominicana en BB- con perspectiva positiva
5. La administración Biden pretende condonar US$4.600 millones de la deuda ucraniana
6. UBS recomienda bonos de $MELI
7. Caputo reconoció que negocia con el FMI los montos para un nuevo programa con Argentina
8. Avanzan negociaciones con el FMI y vuela confianza del consumidor
9. CEO de Boeing pide a sus empleados unión para cambiar el rumbo de la empresa
Human: En forma de lista dame los titulares sobre finanzas
AI: Aquí te presento algunos titulares relacionados con finanzas que se mencionan en el contexto:

* "Dólar

In [50]:
# Invocar la cadena con la memoria
question = "En forma de lista dame los titulares sobre finanzas"
response = chain.invoke(question)

In [51]:
# Guardar el contexto en la memoria
memory.save_context({"question": question}, {"response": response})

In [52]:
# Mostrar el contenido de la memoria
print(memory.buffer)  # Mostrar el historial de la conversación


Human: En forma de lista dame los titulares sobre finanzas
AI: A continuación, te proporciono una lista de titulares relacionados con finanzas:

1. Dólar hoy 22 de noviembre: Peso pierde terreno tras datos de inflación y PIB en México
2. Dólar hoy 21 de noviembre: Peso mexicano encabeza pérdidas entre las mayores divisas
3. UU. sanciona a Gazprombank para restringir exportaciones de gas ruso
4. Fitch Ratings mantiene a República Dominicana en BB- con perspectiva positiva
5. La administración Biden pretende condonar US$4.600 millones de la deuda ucraniana
6. UBS recomienda bonos de $MELI
7. Caputo reconoció que negocia con el FMI los montos para un nuevo programa con Argentina
8. Avanzan negociaciones con el FMI y vuela confianza del consumidor
9. CEO de Boeing pide a sus empleados unión para cambiar el rumbo de la empresa
Human: En forma de lista dame los titulares sobre finanzas
AI: Aquí te presento algunos titulares relacionados con finanzas que se mencionan en el contexto:

* "Dólar